# 코딩테스트 연습 > 스택,큐 > 다리를 지나는 트럭

본 문제의 저작권은 주식회사 프로그래머스에 있습니다. 문제를 풀고 싶으시다면 해당 회사의 웹사이트로 가시면 됩니다.

## 문제 설명
트럭 여러 대가 강을 가로지르는 일 차선 다리를 정해진 순으로 건너려 합니다. 모든 트럭이 다리를 건너려면 최소 몇 초가 걸리는지 알아내야 합니다. 트럭은 1초에 1만큼 움직이며, 다리 길이는 bridge_length이고 다리는 무게 weight까지 견딥니다.
※ 트럭이 다리에 완전히 오르지 않은 경우, 이 트럭의 무게는 고려하지 않습니다.

예를 들어, 길이가 2이고 10kg 무게를 견디는 다리가 있습니다. 무게가 [7, 4, 5, 6]kg인 트럭이 순서대로 최단 시간 안에 다리를 건너려면 다음과 같이 건너야 합니다.

|경과 시간|다리를 지난 트럭|다리를 건너는 트럭|대기 트럭|
|---|---|---|---|
|0|[]|[]|[7,4,5,6]|
|1~2|[]|[7]|[4,5,6]|
|3|[7]|[4]|[5,6]|
|4|[7]|[4,5]|[6]|
|5|[7,4]|[5]|[6]|
|6~7|[7,4,5]|[6]|[]|
|8|[7,4,5,6]|[]|[]|

따라서, 모든 트럭이 다리를 지나려면 최소 8초가 걸립니다.

solution 함수의 매개변수로 다리 길이 bridge_length, 다리가 견딜 수 있는 무게 weight, 트럭별 무게 truck_weights가 주어집니다. 이때 모든 트럭이 다리를 건너려면 최소 몇 초가 걸리는지 return 하도록 solution 함수를 완성하세요.

## 제한 조건
- bridge_length는 1 이상 10,000 이하입니다.
- weight는 1 이상 10,000 이하입니다.
- truck_weights의 길이는 1 이상 10,000 이하입니다.
- 모든 트럭의 무게는 1 이상 weight 이하입니다.

## 입출력 예
|bridge_length|weight|truck_weights|return|
|---|---|---|---|
|2|10|[7,4,5,6]|8|
|100|100|[10]|101|
|100|100|[10,10,10,10,10,10,10,10,10,10]|110|

### 아이디어

단순무식하게 트럭을 하나씩 다리 위에 올려서 비교해가면서 전진 시킬 것입니다.

뎈을 활용해서 로테이션을 돌리면서 새로 들어오는 트럭과 다리 위의 트럭들의 무게를 비교해서 올립니다.

더 이상 다리에 새로운 트럭이 올라가지 못하면 한 번에 잡아 당겨서 loop를 줄입니다.

In [1]:
import numpy as np
from collections import deque

In [215]:
length = 7
weight = 25
trucks = [9,3,20,17,5,3,4,7,2,10]
return_ = 110

무게가 0인 트럭은 없을테니, 빈 자리를 0이라고 하자.

트럭을 덱으로 묶은 뒤, 하나씩 다리 위에 올리는 것이다. 바로 

In [216]:
trucks = deque(trucks)

In [217]:
bridge = deque(np.zeros(length))
bridge[length - 1] = 1

In [218]:
cnt = 0

In [219]:
print('trucks', trucks)
print('bridge', bridge)
print('cnt', cnt)

trucks deque([9, 3, 20, 17, 5, 3, 4, 7, 2, 10])
bridge deque([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1])
cnt 0


In [220]:
while np.array(bridge).sum() != 0.: #트럭이 전부 다리 위에 오른 상태에서, 다리 위의 무게 합이 0이 되면 멈춘다.
    
    # 맨 끝에 트럭을 올리느냐 안 올리느냐 결정하는 코드다.
    if (np.array(bridge).sum() - bridge[length-1])+ trucks[0] <= weight:
        # 다음에 나갈 트럭의 무게를 뺀 다리 위에 있는 무게와 대기 1번 트럭의 무게를 합쳐도 웨이트 보다 작다면
        bridge.rotate(1) # 한칸씩 전진하고
        bridge[0] = trucks.popleft() # 대기 1번 트럭을 트럭스에서 빼고, 브릿지 0번에 넣어준다.
        trucks.append(0)

    else: # 무게가 초과한다면
        bridge.rotate(1) # 한칸씩 전진하고
        bridge[0] = 0 # 빈 칸을 넣어준다.
    
    cnt+=1
    
    # 양끝단이 둘 다 0이라면...맨 앞 놈이 나가기 전에는 뒤 트럭이 들어오지 못한다는 뜻입니다. 이걸 당겨버립시다. 그럼 시간초과 타파.
    try:
        if bridge[0] == 0 and bridge[length -1] == 0: # 다리의 양 끝단이 0이면
            # 확 당겨야 하는데. 일단 0이 아닌 위치를 찾자
            loc = np.where(np.array(bridge) > 0)
            print('       bridge', bridge)
            print('       location', np.where(np.array(bridge) > 0))

            # 다리의 길이에서 가장 앞에 있는 차의 인덱스를 빼준다.
            empty = length - int(loc[0][-1:])

            bridge.rotate(empty - 1) # 어차피 앞에도 0이니 5번을 돌려준다. 급발진.

            # 한 번에 당겨줬으니 다 더해준다.
            cnt += (empty - 1)
    except:
        pass
        
    
    print('trucks', trucks)
    print('bridge', bridge)
    print('cnt', cnt)
    print('-'*80)

trucks deque([3, 20, 17, 5, 3, 4, 7, 2, 10, 0])
bridge deque([9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
cnt 1
--------------------------------------------------------------------------------
trucks deque([20, 17, 5, 3, 4, 7, 2, 10, 0, 0])
bridge deque([3, 9, 0.0, 0.0, 0.0, 0.0, 0.0])
cnt 2
--------------------------------------------------------------------------------
       bridge deque([0, 3, 9, 0.0, 0.0, 0.0, 0.0])
       location (array([1, 2], dtype=int64),)
trucks deque([20, 17, 5, 3, 4, 7, 2, 10, 0, 0])
bridge deque([0.0, 0.0, 0.0, 0.0, 0, 3, 9])
cnt 7
--------------------------------------------------------------------------------
trucks deque([17, 5, 3, 4, 7, 2, 10, 0, 0, 0])
bridge deque([20, 0.0, 0.0, 0.0, 0.0, 0, 3])
cnt 8
--------------------------------------------------------------------------------
       bridge deque([0, 20, 0.0, 0.0, 0.0, 0.0, 0])
       location (array([1], dtype=int64),)
trucks deque([17, 5, 3, 4, 7, 2, 10, 0, 0, 0])
bridge deque([0.0, 0.0, 0.0, 0.0, 0, 0,